In [ ]:
from IPython.display import HTML
from matplotlib_inline.backend_inline import set_matplotlib_formats

# Use inline backend so CSS scaling works on PNG/SVG outputs
set_matplotlib_formats('png')

HTML("""
<style>
/* =======================
   Matplotlib figure sizing
   ======================= */

/* Images (PNG/retina) */
.jp-RenderedImage img,
.jp-OutputArea-output img {
  width: 100% !important;
  height: auto !important;
  max-width: 100% !important;
  display: block;
  box-sizing: border-box;
}

/* SVG outputs */
.jp-RenderedSVG svg,
.jp-OutputArea-output svg {
  width: 100% !important;
  height: auto !important;
}

/* ipympl / canvas-backed figures (in case %matplotlib widget is used) */
.jupyter-matplotlib canvas.mpl-canvas,
.jp-OutputArea-output canvas.mpl-canvas,
.jp-OutputArea-output canvas {
  width: 100% !important;
  height: auto !important;
  max-width: 100% !important;
}

/* Reset container widths */
.jp-OutputArea-output .matplotlib-figure,
.jp-RenderedHTMLCommon .matplotlib-figure,
.jp-OutputArea-output .output_png {
  width: 100% !important;
}

/* =======================
   ipywidgets styling
   ======================= */

/* Make slider labels bold */
.widget-label {
    font-weight: bold;
}

/* Style ipywidgets buttons */
.jp-OutputArea .widget-button,
button.widget-button {
    font-weight: bold;
    background-color: #1976d2;  /* blue background */
    color: white;               /* white text */
    border: none;
    border-radius: 6px;
    padding: 6px 12px;
    font-size: 14px;
    cursor: pointer;
}

/* Button hover effect */
.jp-OutputArea .widget-button:hover,
button.widget-button:hover {
    background-color: #1259a4;  /* darker blue */
}
</style>
""")

# Phasing Our Ears 👂

## 🎯 Learning goals  

- **Understand** how a wave signal’s **angle of arrival** creates phase differences across an array.  
- **Visualize** element phases and coherent summation using **phasors**.  
- **Explore beamforming** as phasor summation that can be steered to reveal source directions.  


## **Mapping Wave Source Direction with the Buoy Array** 📡🌊  

### **Revisiting What We've Learned**  

So far, we've learned how to **focus the energy** of our wave into a **narrow beam** with an antenna array and how to **measure range** (the distance between the radar and the target) using pulsed signals. But there’s still one crucial piece we haven't really considered: **receiving** waves with our antenna array. 📶  How can we determine the **direction of incoming waves**? This is the key to **mapping our environment in 2D** and creating our first **radar images**! 🗺️  

### **Understanding the Buoy Array in Listening Mode** 👂

Imagine our **buoy array** is now **passively listening** to incoming waves. We’re not actively wiggling the buoys to generate waves; instead, we’re **waiting for a wave** to arrive. 🚤 

Our goal is to determine **the direction** from which this wave is coming so we can construct an **angular map of our surroundings**. 🎯  Moreover, when we combine the angle of arrival with out range measurement, we can **map out the 2D location** of surrounding objects on the surface of the lake. 📍

When a **wavefront** reaches our buoys, it makes them **rock up and down** according to its **frequency (rhythm) and amplitude (height)**. Now, suppose we **measure the height of each buoy** relative to the still water surface at each moment in time. What insights can we extract? 🤔  

---

### **How the Arrival Angle Affects the Buoy Motion** 📐 

#### **📍 Case 1: The Wavefront is Head-On**  

If a wave approaches **directly towards the array** (i.e., from a direction perpendicular to the array), its wavefront **aligns perfectly** with the buoy array. In this case, **all buoys rock back and forth in sync**, experiencing **identical motion**.  

#### **📍 Case 2: The Wavefront Arrives at an Angle**  

Now, let’s consider a **wave arriving at an angle**. 📐 Here’s what happens:  
- The **buoy at one edge** of the array encounters the wave **first** and begins rocking. 🛟🌊  
- A short moment later, the **next buoy** in line starts rocking. 🛟⏱️  
- This continues **sequentially across the entire array**. 🛟➡️🛟➡️🛟🌊  

Now, all buoys still oscillate with the **same frequency and amplitude**, but **each is slightly out of phase** with its neighbors. 🔄  The **phase difference** between them depends on the **wavefront's angle** relative to the array.

To see the most extreme case, imagine a wave arriving **from the side**. If the buoys are placed half-wavelength apart, the phase difference between adjacent buoys would reach exactly **half a wavelength** (or 180 degrees), as the wave must travel the full **element spacing** before reaching the next buoy. 📏  

---

### **Visualizing the Effect of Arrival Angle**  

Imagine a speed boat is crusing some distance away from our buyos. 🚤 After some time, the waves generated by the speed boat arrive at our array, let's observe what happens. 🌊

Adjust the **angle of the incoming wavefront** and observe how it **impacts the motion of each buoy**. Here we see that **phasors** are incredibly useful for visualizing the **phase differences between elements**. ⚡ The animation displays the **phasor representation** of the wave as captured by each buoy, illustrating the **phase shifts across the array**. 🎛️  

In [ ]:
def build_wavefront_array_sim():
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import (
        FloatSlider, IntSlider, Play, IntSlider as IS, jslink,
        HBox, VBox, Layout, Output, Label, Button, Dropdown
    )
    from IPython.display import display, clear_output
    import os
    from datetime import datetime

    # -----------------------
    # Fixed simulation params
    # -----------------------
    wavelength    = 1.0     # m
    speed         = 1.0     # m/s
    k             = 2.0 * np.pi / wavelength
    T             = wavelength / speed   # period (s)
    dt_frame      = T / 20.0             # time step per frame
    FRAMES        = 200                  # timeline length
    INTERVAL_MS   = 50                   # Play interval (ms)
    N_elements    = 4                    # buoys / array elements
    Nx, Ny        = 200, 200             # grid resolution
    space_extent  = 5.0 * wavelength     # x in [-E, E], y in [0, 2E]
    DPI           = 96
    # element positions along x at y=0, spacing λ/2
    d = wavelength / 2.0
    positions = np.linspace(-(N_elements-1)/2 * d, (N_elements-1)/2 * d, N_elements)

    # -----------------------
    # Widgets (angles are w.r.t. y-axis, like your original)
    # -----------------------
    angle1 = FloatSlider(
        value=25.0, min=-80.0, max=80.0, step=0.5,
        description="Angle 1 [°]", continuous_update=False
    )
    angle1.style.description_width = 'initial'

    amp1 = FloatSlider(
        value=1.0, min=0.0, max=2.0, step=0.05,
        description="Amp 1", continuous_update=False
    )
    amp1.style.description_width = 'initial'

    angle2 = FloatSlider(
        value=-15.0, min=-80.0, max=80.0, step=0.5,
        description="Angle 2 [°]", continuous_update=False
    )
    angle2.style.description_width = 'initial'

    amp2 = FloatSlider(
        value=0.0, min=0.0, max=2.0, step=0.05,
        description="Amp 2", continuous_update=False
    )
    amp2.style.description_width = 'initial'

    play = Play(value=0, min=0, max=FRAMES, step=1, interval=INTERVAL_MS)
    t_slider = IS(value=0, min=0, max=FRAMES, step=1, description="Time")
    jslink((play, 'value'), (t_slider, 'value'))

    readout = Label()

    # Reset & Save buttons
    btn_reset = Button(description='Reset', tooltip='Restore defaults', button_style='')
    btn_save  = Button(description='Save PNG', tooltip='Save current figures to outputs/', button_style='')
    
    cmap_dd = Dropdown(options=['jet', 'viridis', 'plasma', 'gray', 'coolwarm'], 
                       value='jet', description='Colormap:', 
                       style={'description_width':'initial'})

    # Two rows of controls
    row1 = HBox([angle1, amp1, angle2, amp2],
                layout=Layout(align_items='center', column_gap='12px'))
    row2 = HBox([play, t_slider, btn_reset, btn_save, cmap_dd],
                layout=Layout(align_items='center', column_gap='12px'))
    controls = VBox([row1, row2])

    # -----------------------
    # Spatial grid (fixed)
    # -----------------------
    x = np.linspace(-space_extent, space_extent, Nx)
    y = np.linspace(0.0, 2.0*space_extent, Ny)
    X, Y = np.meshgrid(x, y, indexing='xy')

    # -----------------------
    # Outputs & Figures
    # -----------------------
    out_img     = Output()
    out_phasors = Output()
    out_series  = Output()

    # Wavefront figure
    fig_img, ax_img = plt.subplots(figsize=(7.5, 6), dpi=DPI)
    im = ax_img.imshow(np.zeros((Ny, Nx)), extent=[-space_extent, space_extent, 0, 2*space_extent],
                       origin='lower', cmap=cmap_dd.value, animated=False, vmin=-1.0, vmax=1.0)
    cbar = fig_img.colorbar(im, ax=ax_img, label='Wave Height')
    ax_img.set_title("Planar Wavefront(s) Arriving")
    ax_img.set_xlabel("X Position")
    ax_img.set_ylabel("Y Position")
    # buoys
    ax_img.scatter(positions, np.zeros_like(positions), label="Buoy locations",
                   color="black", s=50, marker='o')
    ax_img.legend(loc='upper right', frameon=True)
    fig_img.tight_layout()
    plt.close(fig_img)  # avoid duplicate static snapshot

    # Phasors figure (one axis per buoy)
    fig_ph, axes_ph = plt.subplots(1, N_elements, figsize=(12, 3), dpi=DPI)
    if N_elements == 1:
        axes_ph = [axes_ph]
    ph_quiv = []
    for i, ax in enumerate(axes_ph):
        q = ax.quiver(
            0, 0, 0, 0,
            angles='xy', scale_units='xy', scale=1, pivot='tail', color='r'
        )
        ax.set_xlim(-2.0, 2.0)
        ax.set_ylim(-2.0, 2.0)
        ax.set_aspect('equal', adjustable='box')
        ax.set_title(f"Phasor buoy #{i+1}")
        ax.set_xlabel("I")
        ax.set_ylabel("Q")
        ax.grid(True, linestyle='--', linewidth=0.5)
        ph_quiv.append(q)
        fig_ph.tight_layout()
        plt.close(fig_ph)

    # Time series figure
    fig_ts, ax_ts = plt.subplots(figsize=(7.5, 3.2), dpi=DPI)
    time_series = np.arange(FRAMES+1) * dt_frame
    ts_lines = [ax_ts.plot([], [], label=f"Buoy {i+1}")[0] for i in range(N_elements)]
    ax_ts.set_xlim(time_series[0], time_series[-1] if len(time_series) > 1 else 1.0)
    ax_ts.set_ylim(-2.0, 2.0)
    ax_ts.set_title("Buoy Signals Over Time")
    ax_ts.set_xlabel("Time [s]")
    ax_ts.set_ylabel("Wave Height")
    ax_ts.legend(loc='upper right', ncols=min(2, N_elements))
    fig_ts.tight_layout()
    plt.close(fig_ts)

    # Display once
    with out_img:
        clear_output(wait=True); display(fig_img)
    with out_phasors:
        clear_output(wait=True); display(fig_ph)
    with out_series:
        clear_output(wait=True); display(fig_ts)

    # -----------------------
    # State & helpers
    # -----------------------
    state = {
        "buoy_signals": np.zeros((N_elements, FRAMES+1), dtype=float),
        "angles_rad": (np.radians(angle1.value), np.radians(angle2.value)),
        "amps": (amp1.value, amp2.value),
    }

    def _precompute_buoy_signals():
        """Precompute each buoy's time series up to FRAMES for the current angles/amplitudes."""
        ang1, ang2 = state["angles_rad"]
        A1, A2 = state["amps"]
        for i, x_i in enumerate(positions):
            phi1_x = k * x_i * np.sin(ang1)
            phi2_x = k * x_i * np.sin(ang2)
            # time phase increments: 2π * t / T
            omega_t = 2.0 * np.pi * time_series / T
            s1 = A1 * np.sin(phi1_x + omega_t)
            s2 = A2 * np.sin(phi2_x + omega_t)
            state["buoy_signals"][i, :] = s1 + s2

        # update y-limits based on amplitude
        A_sum = max(0.5, state["amps"][0] + state["amps"][1])
        ax_ts.set_ylim(-1.1 * A_sum, 1.1 * A_sum)

        # update phasor axes limits too
        for ax in axes_ph:
            ax.set_xlim(-1.2 * A_sum, 1.2 * A_sum)
            ax.set_ylim(-1.2 * A_sum, 1.2 * A_sum)

    def _wavefront_field(t_now):
        """Compute instantaneous field over the grid."""
        ang1, ang2 = state["angles_rad"]
        A1, A2 = state["amps"]
        phi1 = k * (X * np.sin(ang1) + Y * np.cos(ang1)) + 2.0 * np.pi * t_now / T
        phi2 = k * (X * np.sin(ang2) + Y * np.cos(ang2)) + 2.0 * np.pi * t_now / T
        return A1 * np.sin(phi1) + A2 * np.sin(phi2)

    def _draw(frame_idx: int):
        """Update all three views for the given time index."""
        frame_idx = int(np.clip(frame_idx, 0, FRAMES))
        t_now = frame_idx * dt_frame

        # --- Update wavefront image ---
        field = _wavefront_field(t_now)
        im.set_array(field)
        # dynamic color scale around combined amplitude
        v = max(0.5, state["amps"][0] + state["amps"][1])
        im.set_clim(-v, v)

        # --- Update time series (show up to current frame) ---
        for i, line in enumerate(ts_lines):
            line.set_data(time_series[:frame_idx+1], state["buoy_signals"][i, :frame_idx+1])

        # --- Update phasors (I/Q at each buoy) ---
        ang1, ang2 = state["angles_rad"]
        A1, A2     = state["amps"]
        for i, x_i in enumerate(positions):
            # y=0 for buoys ⇒ phase depends on x only
            phi1 = k * x_i * np.sin(ang1) + 2.0 * np.pi * t_now / T
            phi2 = k * x_i * np.sin(ang2) + 2.0 * np.pi * t_now / T
            ph = A1 * np.exp(1j * phi1) + A2 * np.exp(1j * phi2)
            ph_quiv[i].set_UVC(np.array([np.real(ph)]), np.array([np.imag(ph)]))

        # Render updated figures
        with out_img:
            clear_output(wait=True); display(fig_img)
        with out_series:
            clear_output(wait=True); display(fig_ts)
        with out_phasors:
            clear_output(wait=True); display(fig_ph)

        # Readout
        deg1 = np.degrees(ang1); deg2 = np.degrees(ang2)
        readout.value = (f"λ = {wavelength:.2f} m | k = {k:.2f} 1/m | T = {T:.2f} s | "
                         f"Angle1 = {deg1:+.1f}°, Amp1 = {A1:.2f} | "
                         f"Angle2 = {deg2:+.1f}°, Amp2 = {A2:.2f}")

    def _on_param_change(_=None):
        # store angles & amplitudes from sliders
        state["angles_rad"] = (np.radians(angle1.value), np.radians(angle2.value))
        state["amps"] = (float(amp1.value), float(amp2.value))
        _precompute_buoy_signals()
        _draw(t_slider.value)

    def _on_reset(_):
        angle1.value = 25.0
        amp1.value   = 1.0
        angle2.value = -15.0
        amp2.value   = 0.0
        t_slider.value = 0

    def _ensure_outputs_dir():
        out_dir = os.path.join('outputs')
        os.makedirs(out_dir, exist_ok=True)
        return out_dir

    def _on_save(_):
        out_dir = _ensure_outputs_dir()
        ts = datetime.now().strftime('%Y%m%d_%H%M%S')
        base = f'wavefront_array_{ts}'
        fig_img.savefig(os.path.join(out_dir, base + '_field.png'), dpi=DPI, bbox_inches='tight')
        fig_ph.savefig(os.path.join(out_dir, base + '_phasors.png'), dpi=DPI, bbox_inches='tight')
        fig_ts.savefig(os.path.join(out_dir, base + '_timeseries.png'), dpi=DPI, bbox_inches='tight')

    # Wire up controls
    for w in (angle1, amp1, angle2, amp2):
        w.observe(_on_param_change, names='value')
    t_slider.observe(lambda ch: _draw(ch["new"]), names='value')
    def _update_colormap(_):
        im.set_cmap(cmap_dd.value)
        _draw(t_slider.value)
    cmap_dd.observe(_update_colormap, names='value')
    btn_reset.on_click(_on_reset)
    btn_save.on_click(_on_save)

    # Initial compute & draw
    _on_param_change()

    # Return composed UI
    return VBox([
        controls,
        readout,
        out_img,
        out_phasors,
        out_series
    ])

# Example:
ui = build_wavefront_array_sim()
display(ui)

### 🧪 Try this
- Set **Angle 1** to 0° and increase **Amp 1**; all buoys rock in sync.
- Move **Angle 1** toward ±80°; watch phases spread across buoys.
- Add a second source: set **Amp 2 > 0** and choose a different angle; note the mixed phasors.


### **The Principle of Direction Finding** 📡🎯  

By merely **comparing the phases** of neighboring buoys, we can **deduce the wavefront's direction**:  
- If all buoys **rock in sync**, the wave is coming **from straight ahead**. 🌊  
- If they are completely **out of sync**, with a **phase difference of half a wavelength**, the wave is arriving **from the side**. 📏  

This is the foundation of **direction finding**, a fundamental concept in **radar and sonar arrays**. 🔍  

---

### **What Happens with Multiple Wave Sources?** 🌊🌊

But what if **multiple sources** generate waves **at the same time**? 🤔 The situation quickly becomes **more complex**. When different waves **superimpose**, we can no longer **separate them neatly** by just comparing **phase differences**. Instead, what reaches us is a **mixture of contributions from many directions**—like the choppy water when a fleet of speedboats races past at once. 🚤🚤🚤🌊

You can experiment with the above simulation by **adding a second target** at a different angle and **see how the resulting wavefront looks**. Rerun the code after setting a non-zero value for the amplitude of the second target. You can also test adjusting the amplitude and azimuth angles of both targets.

At first glance, two overlapping wavefronts might seem **incredibly confusing**. How could we possibly **untangle such a mess**? 😵‍💫 Don’t worry, this apparent chaos is exactly what beamforming on receive is designed to sort out. 

### **Beam Steering to the Rescue** 🎯📡🎧

In the previous lesson, you saw how we can **steer the antenna beam** during **transmission** by applying **phase shifts** across the array — shaping the outgoing wave so it adds up strongest in the desired direction.

Here’s the cool part:  
**The exact same principle applies when receiving.** 🎧 By shifting the phase of the **incoming echoes** before we combine them, we can make the signal add up most strongly from **any direction we choose**. It’s like turning the ears of the array toward the sound we want to hear. 👂

And there's more:  
When we **record the signals at each antenna element**, we don't have to commit to a direction right away. We can apply the phase shifts **after the fact** — even after the wave has passed — and "digitally" steer the beam in **any direction we want**. 🤯

> **Think of it like this:**  
> During transmission, we shape the outgoing beam, which controls what targets we illuminate.  
> During reception, we **listen more carefully** in selected directions — even multiple directions at once.

Let’s explore how this works with an analogy. Imagine that there’s a pencil ✏️ attached to the side of each buoy. Behind each buoy, there’s a moving sheet of paper 📜. As the buoy rocks up and down, it plots the height of the wave on the moving paper—just like a polygraph does! This corresponds to each **antenna element** in a **phased array** radar recording the **voltage over time** ⚡⏱️. With a recording like this from each buoy, we now have all the data we need to turn the ears of the array in any direction we want. 🎧   

---

## 📏 How We "Catch" Waves Coming from a Certain Direction

Instead of **sending** a wave in a particular direction, what if we want to **listen** from a specific direction? 👂  

When a wave arrives at our antenna array from an angle, it doesn't hit all the elements at the same time. It reaches one end first, then the next, and so on — just like a wave hitting a line of buoys on the water.

This means that each antenna element "hears" the wave **slightly out of sync** with the others. If we simply add up these signals as-is, they won't line up — the wave peaks and troughs won’t match — so they won’t add up in the same phase. 

This is exactly what happens to our transmit beam at angles **away from the mainlobe**, where the waves do not combine coherently.

### 🎯 So what do we do?

We **digitally shift** each signal, nudging them ever so slightly forward or backward in time, so that the wave peaks **line up perfectly**. When we do this just right:

- If the wave really **came from the direction we expected**, all the signals line up and **add up strongly** — like stacking identical waves on top of each other. 🌊🌊🌊  
- If the wave came from **a different direction**, our timing correction is "wrong," and the waves don’t line up — so they mostly **cancel each other out**.

What does **digitally shifting** actually mean, you might ask? With the recorded wave from each array element, we can build the corresponding **phasors**. Then we simply rotate the phasor to a new angle—clean and simple!

### 🛠️ In practice:

1. We figure out **how much delay** (or shift) each antenna element needs to sync up with the others  
2. We apply those shifts to the signals  
3. We **add up all the adjusted signals**

This lets us to "focus our ears" in a certain direction and ignore others, like turning a directional microphone toward a sound source. 🎙️👂📡

> **In short:** We realign the signals so that waves from the direction we care about reinforce each other — and everything else fades into the background.


### Radar Engineer's Sweetheart: Phasors ❤️

**Phasors** are one of a radar engineer’s most beloved tools, and beamforming is a perfect example of why. 💖

Imagine each antenna element receives the same wave, but **slightly out of sync** because the wavefront reaches them at different times. If we represent each element’s signal as a **phasor** — a little arrow spinning around a circle — then each arrow points in a **different direction**, depending on when that part of the wave was received. Just as we saw in the first simulation of this notebook.

Now here’s the magic of beamforming:

1. We **rotate each phasor** (adjust its phase) so that they all **point in the same direction**
2. Then we **add them together**

If our phase adjustments match the true direction the wave came from, all the arrows line up — and the result is a **very long arrow**. That’s a strong signal! 📈

But if we apply the **wrong phase shifts**, the phasors don’t align. They point in different directions, and when we add them up, they **partially cancel each other out**. The result is a short, messy arrow — a weak signal. 📉

So beamforming is really about **rotating phasors** just right so they all point in the same direction. 🔄🌀📐 When they do, the signal stands out clearly — like voices singing in harmony instead of shouting over each other. 🎶🎤

In [ ]:
def build_phasor_summation_sim():
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import (
        IntSlider, FloatSlider, Play, IntSlider as IS, jslink,
        HBox, VBox, Layout, Output, Label, Button
    )
    from IPython.display import display, clear_output
    import os
    from datetime import datetime

    # -----------------------
    # Widgets (linear phase only)
    # -----------------------
    num_elements = IntSlider(value=6, min=2, max=24, step=1,
                             description="Elements", continuous_update=False)
    num_elements.style.description_width = 'initial'

    phase_step_deg = FloatSlider(value=30.0, min=-180.0, max=180.0, step=1.0,
                                 description="Δφ / elem [°]", continuous_update=False)
    phase_step_deg.style.description_width = 'initial'

    play = Play(value=0, min=0, max=2, step=1, interval=1500)
    frame_slider = IS(value=0, min=0, max=2, step=1, description="Stage")
    jslink((play, 'value'), (frame_slider, 'value'))

    readout = Label()

    # Reset & Save buttons
    btn_reset = Button(description='Reset', tooltip='Restore defaults', button_style='')
    btn_save  = Button(description='Save PNG', tooltip='Save current figure to outputs/', button_style='')

    # Two rows of controls
    row1 = HBox([num_elements, phase_step_deg],
                layout=Layout(align_items='center', column_gap='10px'))
    row2 = HBox([play, frame_slider, btn_reset, btn_save],
                layout=Layout(align_items='center', column_gap='10px'))

    # -----------------------
    # Output / Figure
    # -----------------------
    out = Output()
    fig, ax = plt.subplots(figsize=(7.5, 7.5), dpi=96)
    fig.tight_layout()
    plt.close(fig)  # prevent duplicate static snapshot in Voila

    # -----------------------
    # State
    # -----------------------
    state = {
        "phases_rad": None,  # per-element received phases (linear ramp)
        "colors": None,      # per-element colors
    }

    # -----------------------
    # Helpers
    # -----------------------
    def _gen_colors(n):
        tab10 = plt.get_cmap("tab10").colors
        return [tab10[i % len(tab10)] for i in range(n)]

    def _rebuild_phases():
        n = int(num_elements.value)
        step = np.deg2rad(float(phase_step_deg.value))
        phases = np.arange(n) * step
        state["phases_rad"] = phases
        state["colors"] = _gen_colors(n)

    def _arrow(ax, x0, y0, dx, dy, color, alpha=1.0, label=None,
               head_width=0.2, head_length=0.3, lw=1.5):
        ax.arrow(x0, y0, dx, dy, width=0.0, head_width=head_width, head_length=head_length,
                 fc=color, ec=color, alpha=alpha, length_includes_head=True, lw=lw,
                 label=label)

    # -----------------------
    # Draw per stage (0,1,2)
    # -----------------------
    def _draw(stage: int):
        ax.clear()
        n = int(num_elements.value)
        phases = state["phases_rad"]
        colors = state["colors"]

        # Axes + limits
        lim = max(2.0, 1.2 * n)
        ax.set_xlim(-lim, lim)
        ax.set_ylim(-lim, lim)
        ax.axhline(0, color='black', lw=0.5)
        ax.axvline(0, color='black', lw=0.5)
        ax.set_xlabel("I")
        ax.set_ylabel("Q")
        ax.set_aspect('equal', adjustable='box')
        ax.grid(True, linestyle='dotted', alpha=0.7)

        if stage == 0:  # Received phasors (NO resultant)
            for i, (phi, col) in enumerate(zip(phases, colors)):
                _arrow(ax, 0, 0, np.cos(phi), np.sin(phi), color=col, label=f'Element {i+1}')
            ax.legend(loc='upper left', ncols=2, frameon=True)
            title = "Received Phasors (linear phase ramp)"
            readout.value = f"N = {n} | Δφ/elem = {phase_step_deg.value:+.1f}°"

        elif stage == 1:  # After phase correction (aligned) — NO resultant
            for i, col in enumerate(colors):
                _arrow(ax, 0, 0, 1, 0, color=col, alpha=0.65, label=f'Element {i+1}')
            ax.legend(loc='upper left', ncols=2, frameon=True)
            title = "After Phase Correction (aligned)"
            readout.value = f"N = {n} | Phases aligned"

        else:  # stage == 2, Coherent Summation (head-to-tail) — show resultant
            x0 = 0.0
            for i, col in enumerate(colors):
                _arrow(ax, x0, 0, 1, 0, color=col, label=f'Element {i+1}')
                x0 += 1.0
            # Resultant arrow from origin
            _arrow(ax, 0, 0, n, 0, color='k', lw=2.0,
                   head_width=0.25, head_length=0.35, label='Resultant')
            ax.legend(loc='upper left', ncols=2, frameon=True)
            title = "Coherent Summation"
            readout.value = f"N = {n} | Resultant = {n:.2f} ∠ 0°"

        ax.set_title(title)

        with out:
            clear_output(wait=True)
            display(fig)

    # -----------------------
    # Wiring
    # -----------------------
    def _on_change(_=None):
        _rebuild_phases()
        _draw(frame_slider.value)

    def _on_reset(_):
        num_elements.value = 6
        phase_step_deg.value = 30.0
        frame_slider.value = 0

    def _ensure_outputs_dir():
        out_dir = os.path.join('outputs')
        os.makedirs(out_dir, exist_ok=True)
        return out_dir

    def _on_save(_):
        out_dir = _ensure_outputs_dir()
        ts = datetime.now().strftime('%Y%m%d_%H%M%S')
        fig.savefig(os.path.join(out_dir, f'phasor_summation_{ts}.png'), dpi=96, bbox_inches='tight')

    # Observe widgets
    num_elements.observe(_on_change, names='value')
    phase_step_deg.observe(_on_change, names='value')
    frame_slider.observe(lambda ch: _draw(ch["new"]), names='value')
    btn_reset.on_click(_on_reset)
    btn_save.on_click(_on_save)

    # Initial setup & draw
    _rebuild_phases()
    _draw(frame_slider.value)

    return VBox([row1, row2, readout, out])

phasors = build_phasor_summation_sim()
display(phasors)

### 🧪 Try this
- Step through stages with the play control to see receive, align, and sum.
- Increase **Elements**; compare resultant length in stage 2.
- Try negative vs positive **Δφ / elem**; note the rotation direction.


## Scanning Our Surroundings via Beamforming 🎯 

To map out the incoming waves from entire angular sector in front of us, we can simply repeat this process:  

1️⃣ **For each direction (azimuth angle)**, compute the required phase shifts between the elements.  
2️⃣ **Align the signals** (phasors) of each element accordingly.  
3️⃣ **Sum them together** to reinforce the signals from the analyzed azimuth direction.  

This process is analogous to using a **magnifying glass** 🔎 and turning it to examine targets at different directions. The summation acts as the **focusing operation**, just as a lens gathers light waves across its surface into a single point.  

**But here’s the cool part**:  We **don’t need to physically rotate** our "magnifying glass" to switch the direction! 🔎 Instead, by applying the **appropriate phase shifts** to the signals recorded by each element, we can **steer our reception beam** in any direction we choose, entirely through **signal processing!** ✨📡  

In the upcoming notebooks, we'll explore how to build a **2D image** combining **range (time-delay)** and **angle** information. This is achieved by performing the scan over multiple time snapshots, each corresponding to a different time delay (i.e., range). The result? A detailed image that reveals **both the distance and angular position** of objects. 🔍✨

## 🎛️ Try It Yourself!  

Experiment and visualize how we can turn the **gaze of our array** via beamforming to map out the directions of the wave echoes below. The animation below illustrates the **beamfroming process for a single time snapshot**, obtained by taking a sample (i.e. **recording the amplitude and phase**) of the incoming wavefront for each element.

🔹 **Start with a single target**—keeping the amplitude of the second target as zero.  
🔹 **Then, add a second target** and observe the changes! Feel free to adjust the target amplitudes, angles, and the array size! 🧪 

If you place the **targets far enough apart in angle**, you will observe **two distinct peaks** in the **beamforming output**, corresponding to the angles where the waves originated. 🎯📡  

You will notice each target also producing **sidelobes** at unwanted locations, just as we had for our transmitted beam pattern. 📶 

In [ ]:
def build_beamforming_sim():
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import (
        FloatSlider, IntSlider, Play, IntSlider as IS, jslink,
        HBox, VBox, Layout, Output, Label, Button, Dropdown
    )
    from IPython.display import display, clear_output
    import os
    from datetime import datetime

    # -----------------------
    # Fixed parameters
    # -----------------------
    wavelength   = 1.0   # m
    k            = 2.0 * np.pi / wavelength
    space_extent = 5.0 * wavelength
    Nx, Ny       = 200, 200             # grid resolution for wavefront snapshot
    DPI          = 96
    ANGLE_MIN_DEG = -80.0
    ANGLE_MAX_DEG =  80.0
    NUM_ANGLES    = 181                 # −80..+80 in 1° steps
    beam_angle_vec = np.linspace(np.radians(ANGLE_MIN_DEG),
                                 np.radians(ANGLE_MAX_DEG),
                                 NUM_ANGLES)
    beam_angle_deg = np.degrees(beam_angle_vec)

    # -----------------------
    # Widgets
    # -----------------------
    ang1 = FloatSlider(value=10.0,  min=-80.0, max=80.0, step=0.5,
                       description="Target 1 [°]", continuous_update=False)
    amp1 = FloatSlider(value=1.0,   min=0.0,  max=2.0,  step=0.05,
                       description="Amp 1",    continuous_update=False)

    ang2 = FloatSlider(value=-35.0, min=-80.0, max=80.0, step=0.5,
                       description="Target 2 [°]", continuous_update=False)
    amp2 = FloatSlider(value=0.0,   min=0.0,  max=2.0,  step=0.05,
                       description="Amp 2",    continuous_update=False)

    n_el = IntSlider(value=10, min=2, max=64, step=1,
                     description="Elements", continuous_update=False)

    play = Play(value=0, min=0, max=NUM_ANGLES-1, step=1, interval=60)
    idx  = IS(value=0, min=0, max=NUM_ANGLES-1, step=1, description="Scan step")
    jslink((play, 'value'), (idx, 'value'))

    readout = Label()

    # Reset & Save buttons
    btn_reset = Button(description='Reset', tooltip='Restore defaults', button_style='')
    btn_save  = Button(description='Save PNG', tooltip='Save current figures to outputs/', button_style='')
    
    cmap_dd = Dropdown(options=['jet', 'viridis', 'plasma', 'gray', 'coolwarm'], 
                       value='jet', description='Colormap:', 
                       style={'description_width':'initial'})

    ctrls_top = HBox([ang1, amp1, ang2, amp2],
                     layout=Layout(align_items='center', column_gap='12px'))
    ctrls_bot = HBox([n_el, play, idx, btn_reset, btn_save, cmap_dd],
                     layout=Layout(align_items='center', column_gap='12px'))

    # -----------------------
    # Spatial grid (fixed)
    # -----------------------
    x = np.linspace(-space_extent, space_extent, Nx)
    y = np.linspace(0.0, 2.0*space_extent, Ny)
    X, Y = np.meshgrid(x, y, indexing='xy')

    # -----------------------
    # Outputs & Figures
    # -----------------------
    out_wave = Output()
    out_beam = Output()

    # Wavefront figure (snapshot at t=0; angles measured relative to y-axis)
    fig_w, ax_w = plt.subplots(figsize=(7.5, 6), dpi=DPI)
    im = ax_w.imshow(np.zeros((Ny, Nx)), extent=[-space_extent, space_extent, 0, 2*space_extent],
                     origin='lower', cmap=cmap_dd.value, vmin=-1.0, vmax=1.0)
    cbar = fig_w.colorbar(im, ax=ax_w, label='Wave Height')
    ax_w.set_title("Incident Planar Wavefront(s)")
    ax_w.set_xlabel("X position")
    ax_w.set_ylabel("Y position")
    fig_w.tight_layout()
    plt.close(fig_w)

    # Beam scan + summed phasor (with head-to-tail element phasors)
    fig_b, (ax_bscan, ax_sum) = plt.subplots(1, 2, figsize=(12, 4.4), dpi=DPI)
    line_beam, = ax_bscan.plot([], [], 'g-', label="Beamformer Output |Σ|")
    current_pt, = ax_bscan.plot([], [], 'ko', ms=5, label='Current')

    # NEW: target location markers (dashed vertical lines)
    t1_v = ax_bscan.axvline(np.nan, color='tab:red',   linestyle='--', linewidth=1.5, label='Target 1 θ')
    t2_v = ax_bscan.axvline(np.nan, color='tab:blue',  linestyle='--', linewidth=1.5, label='Target 2 θ')

    ax_bscan.set_xlim(ANGLE_MIN_DEG, ANGLE_MAX_DEG)
    ax_bscan.set_xlabel("Steering angle [deg]")
    ax_bscan.set_ylabel("Summed magnitude")
    ax_bscan.set_title("Beam Scan")
    ax_bscan.legend(loc='upper right')

    # Right panel: element phasor chain + final summed phasor
    corrected_quivers = []  # per-element arrows (black)
    summed_q = ax_sum.quiver(0, 0, 0, 0, angles='xy', scale_units='xy', scale=1,
                             color='tab:green', pivot='tail', label='Σ (summed)')
    ax_sum.set_xlabel("I"); ax_sum.set_ylabel("Q")
    ax_sum.grid(True, linestyle='--', linewidth=0.5)
    ax_sum.legend(loc='upper left', fontsize='small')
    ax_sum.set_title("Phase-corrected phasor chain at current θ")
    fig_b.tight_layout()
    plt.close(fig_b)

    # Display initial figs
    with out_wave:
        clear_output(wait=True); display(fig_w)
    with out_beam:
        clear_output(wait=True); display(fig_b)

    # -----------------------
    # State & helpers
    # -----------------------
    state = {
        "positions": None,   # element positions (x)
        "received": None,    # complex per-element received phasors (targets)
        "S_all": None,       # complex sum vs angle (precomputed)
        "A_sum": 1.0         # amplitude scale for axes
    }

    def _positions(n):
        d = wavelength / 2.0
        return np.linspace(-(n-1)/2.0 * d, (n-1)/2.0 * d, n)

    def _update_wavefront():
        th1 = np.radians(ang1.value); th2 = np.radians(ang2.value)
        A1  = float(amp1.value);      A2  = float(amp2.value)
        # Angles relative to y-axis: phase = k*(x*sinθ + y*cosθ)
        field = A1 * np.sin(k * (X * np.sin(th1) + Y * np.cos(th1))) + \
                A2 * np.sin(k * (X * np.sin(th2) + Y * np.cos(th2)))
        im.set_array(field)
        vmax = max(0.5, A1 + A2)
        im.set_clim(-vmax, +vmax)

        with out_wave:
            clear_output(wait=True); display(fig_w)

    def _rebuild_chain_axes(n):
        """Recreate the right panel with n element arrows (black) plus the green sum."""
        nonlocal corrected_quivers, summed_q
        ax_sum.cla()
        corrected_quivers = [ax_sum.quiver(0, 0, 0, 0, angles='xy', scale_units='xy',
                                           scale=1, color='k', alpha=0.75, pivot='tail')
                             for _ in range(n)]
        summed_q = ax_sum.quiver(0, 0, 0, 0, angles='xy', scale_units='xy',
                                 scale=1, color='tab:green', pivot='tail', label='Σ (summed)')
        ax_sum.set_xlabel("I"); ax_sum.set_ylabel("Q")
        ax_sum.grid(True, linestyle='--', linewidth=0.5)
        ax_sum.legend(loc='upper left', fontsize='small')
        ax_sum.set_title("Phase-corrected phasor chain at current θ")

    def _update_target_lines():
        """Move dashed vertical lines to current target angles (dim if amplitude = 0)."""
        deg1 = float(ang1.value)
        deg2 = float(ang2.value)
        # set_xdata expects a 2-point x array for a vertical line
        t1_v.set_xdata([deg1, deg1]); t1_v.set_alpha(0.9 if amp1.value > 0 else 0.25)
        t2_v.set_xdata([deg2, deg2]); t2_v.set_alpha(0.9 if amp2.value > 0 else 0.25)

    def _recompute_all():
        """Recompute element responses and full beam scan for current settings."""
        n   = int(n_el.value)
        th1 = np.radians(ang1.value); th2 = np.radians(ang2.value)
        A1  = float(amp1.value);      A2  = float(amp2.value)

        pos = _positions(n)
        p_rx = A1 * np.exp(1j * k * pos * np.sin(th1)) + \
               A2 * np.exp(1j * k * pos * np.sin(th2))

        # Precompute Σ(θ) across all steering angles
        sin_theta = np.sin(beam_angle_vec)[:, None]              # (NUM_ANGLES, 1)
        weights   = np.exp(-1j * k * pos[None, :] * sin_theta)   # (NUM_ANGLES, n)
        S_all     = weights @ p_rx

        state["positions"] = pos
        state["received"]  = p_rx
        state["S_all"]     = S_all
        state["A_sum"]     = max(0.1, A1 + A2)

        # Rebuild chain axes and rescale limits
        _rebuild_chain_axes(n)
        lim = n * state["A_sum"]
        ax_sum.set_xlim(-lim, +lim)
        ax_sum.set_ylim(-lim, +lim)

        # Reset the beam curve to progressive style (empty until drawn)
        line_beam.set_data([], [])
        current_pt.set_data([], [])

        # Update beam plot Y-limits with a safe max
        ymax = np.max(np.abs(S_all))
        ax_bscan.set_ylim(0, max(1.0, ymax * 1.05))

        # Update target markers
        _update_target_lines()

        with out_beam:
            clear_output(wait=True); display(fig_b)

    def _draw(i):
        """Update progressive beam curve and chain at steering index i."""
        i = int(np.clip(i, 0, NUM_ANGLES-1))
        n = int(n_el.value)
        pos = state["positions"]; p_rx = state["received"]; S_all = state["S_all"]
        if pos is None or p_rx is None or S_all is None:
            return

        theta = beam_angle_vec[i]

        # Phase-corrected element phasors at this θ
        w = np.exp(-1j * k * pos * np.sin(theta))
        corrected = p_rx * w

        # Head-to-tail chain
        cumulative = 0.0 + 0.0j
        for j in range(n):
            start_x = np.real(cumulative)
            start_y = np.imag(cumulative)
            corrected_quivers[j].set_offsets(np.array([[start_x, start_y]]))
            corrected_quivers[j].set_UVC(np.real(corrected[j]), np.imag(corrected[j]))
            cumulative += corrected[j]

        # Summed phasor
        summed = cumulative  # == np.sum(corrected)
        summed_q.set_UVC(np.real(summed), np.imag(summed))

        # Progressive beam curve (up to i)
        mags = np.abs(S_all[:i+1])
        line_beam.set_data(beam_angle_deg[:i+1], mags)
        current_pt.set_data([beam_angle_deg[i]], [mags[-1]])

        # Readout
        readout.value = (f"N = {n} | "
                         f"θ₁ = {ang1.value:+.1f}° (A₁={amp1.value:.2f}), "
                         f"θ₂ = {ang2.value:+.1f}° (A₂={amp2.value:.2f}) | "
                         f"Scan θ = {beam_angle_deg[i]:+5.1f}° | |Σ| = {np.abs(summed):.2f}")

        # Refresh figure
        with out_beam:
            clear_output(wait=True); display(fig_b)

    # -----------------------
    # Wiring
    # -----------------------
    def _on_targets_change(_=None):
        _update_wavefront()
        _recompute_all()
        _draw(idx.value)

    def _on_n_change(_=None):
        _recompute_all()
        _draw(idx.value)

    def _on_reset(_):
        ang1.value = 10.0
        amp1.value = 1.0
        ang2.value = -35.0
        amp2.value = 0.0
        n_el.value = 10
        idx.value = 0

    def _ensure_outputs_dir():
        out_dir = os.path.join('outputs')
        os.makedirs(out_dir, exist_ok=True)
        return out_dir

    def _on_save(_):
        out_dir = _ensure_outputs_dir()
        ts = datetime.now().strftime('%Y%m%d_%H%M%S')
        base = f'beamforming_{ts}'
        fig_w.savefig(os.path.join(out_dir, base + '_wave.png'), dpi=DPI, bbox_inches='tight')
        fig_b.savefig(os.path.join(out_dir, base + '_beam.png'), dpi=DPI, bbox_inches='tight')

    idx.observe(lambda ch: _draw(ch["new"]), names='value')
    for w in (ang1, amp1, ang2, amp2):
        w.observe(_on_targets_change, names='value')
    n_el.observe(_on_n_change, names='value')
    def _update_colormap(_):
        im.set_cmap(cmap_dd.value)
        _draw(idx.value)
    cmap_dd.observe(_update_colormap, names='value')
    btn_reset.on_click(_on_reset)
    btn_save.on_click(_on_save)

    # Initial build
    _update_wavefront()
    _recompute_all()
    _draw(idx.value)

    return VBox([ctrls_top, ctrls_bot, readout, out_wave, out_beam])

beamforming = build_beamforming_sim()
display(beamforming)


### 🧪 Try this
- Start with **Amp 2 = 0** and **Elements = 10**; scan and locate the single peak.
- Add a second target: set **Amp 2 > 0** at a different angle; watch for two peaks emerge.
- Increase **the number of elements**; peaks sharpen.
- Move targets close together; notice peak merging.

## Understanding Beamforming Through Phasors ↗️

Beamforming might sound mysterious, but at its heart it’s just **phasors playing teamwork** ⚡➡️⚡.  

In the **first animation**, we sweep across angles and measure how strong the combined signal is.  

👉 When the **scanned angle matches the true target angle(s)**, the wave echoes **line up in phase** and add up into a big peak in the beamformer output 📈.  

The **second animation** opens the box and shows *how beamforming works under the hood*:  

- Each antenna element contributes a **phase-corrected phasor** (**⚫ black arrows**).  
- We **sum them together** to form the **beam output at that angle** (**🟢 green arrow**).  
- The plot on the left tracks the length of this green arrow — that’s the beamformer response at the scanned angle.  

✨ That’s really all there is to it:  
**Adjust the arrow directions (phases) → add them up → read the result**. **Radar signal processing math is really not more complicated than this!** ↗️ ➕ ↖️ It’s all about **adjusting the direction of the arrows** based on phase, and then **adding them together**!  

## 📌 Summary

- **Arrival angle** causes **phase differences** across elements; in-phase means straight ahead.
- **Beamforming**: rotate element phasors to align, then sum; peaks reveal directions.
- Multiple sources appear as **multiple peaks**; sidelobes appear just like on transmit.
